In [16]:
import numpy as np


class Neuron:
    def __init__(self, size):
        self._weights = np.random.rand(size)
        self._previous_changes = [0, 0]
        self._input = 0
        self._output = 0

    def forward(self, x):
        self._input = x
        linear_input = np.dot(x, self._weights)
        self._output = self._sigmoid(linear_input)
        return self._output

    def backward(self, x, loss, epsilon=0.5, alpha=0.2):
        sigmoid_der = self._output * (1 - self._output)
        delta = loss * sigmoid_der
        losses = []
        for i in range(len(self._weights)):
            grad = self._input[i] * delta
            delta_w = epsilon * grad + alpha * self._previous_changes[i]
            self._previous_changes[i] = delta_w
            self._weights[i] += delta_w[0]
            losses.append(self._weights[i] * delta)
        return losses

    @staticmethod
    def _sigmoid(x):
        return 1 / (1 + np.exp(-x))

In [17]:
class Model:
    def __init__(self):
        self.hidden_1 = Neuron(2)
        self.hidden_2 = Neuron(2)
        self.out = Neuron(2)

    def forward(self, x):
        h1_out = self.hidden_1.forward(x)
        h2_out = self.hidden_2.forward(x)
        output = self.out.forward([h1_out, h2_out])
        return output

    def backward(self, x, loss):
        losses = self.out.backward(x, loss)
        self.hidden_1.backward(x, losses[0])
        self.hidden_2.backward(x, losses[1])

    def train(self, data, labels, epochs):
        for epoch in range(epochs):
            for itr in range(len(data)):
                y = self.forward(data[itr])
                self.backward(data[itr], labels[itr] - y)

    def check_error(self, data, labels):
        y = []
        for x in data:
            y.append(self.forward(x))
        print('Error:', self._err(labels, y, 4))

    @staticmethod
    def _err(ideal, actual, count):
        res = 0
        for i in range(count):
            res += (ideal[i][0] - actual[i]) ** 2
        return res / count

In [20]:
model = Model()
data = [[0, 0], [0, 1], [1, 0], [1, 1]]
labels = [[0], [1], [1], [0]]
model.train(data, labels, 50000)
model.check_error(data, labels)

Error: 0.0012425590235285245
